In [ ]:
import os

id = 99
pwd = os.getcwd()
print pwd

new_id = '12345678' + str(id)
new_id = new_id[-4:]
print new_id

In [55]:
#Submit script!!!
import json
import re
import pandas as pd
from finalize_JL import find_neighbors
from finalize_JL import read_mutations_resfile

#Parses dataset .json file and outputs chain to move and input PDB file directory
def json_parser():
####asdf = open("~/ddg-zemu-runs/160315-kyleb_james-backrub-rscript/data/blank_job_dict.json")
    asdf = open("/Users/jameslucas/Kortemme_Rotation/blank_job_dict_copy.json")
    jsonfile = json.load(asdf)

    chaintomove = pd.DataFrame()
    directory = pd.DataFrame()
    index = pd.DataFrame()
    count = 0
    
    for i in jsonfile:
        chaintomove = chaintomove.append([jsonfile[i]["%%chainstomove%%"]])
        directory = directory.append([jsonfile[i]["input_file_list"]])
        index = index.append([count])
        count = count + 1
        
    chaintomove.columns = ['Chaintomove']
    directory.columns = ['PDB File Path']
    index.columns = ['Index']
    
    result = pd.concat([chaintomove, directory], axis=1, join_axes=[chaintomove.index])
    result = pd.concat([index, result], axis=1)
    output = result.set_index('Index')
    return output

#Finds neighbors within 8A and adds position and Chain information to a pandas dataframe
def neighbors_list(pdb_filepath, pdb_file):
    neighbors = find_neighbors("/Users/jameslucas/Kortemme_Rotation/"+pdb_filepath, "/Users/jameslucas/Kortemme_Rotation/"+pdb_file, 8)
    info = pd.DataFrame(columns=('Residue', 'Chain'))
    hold = pd.DataFrame(columns=('Residue', 'Chain'))

    for i in neighbors:
        string_parse = re.sub("[(),']",'', str(i))
        for s in string_parse.split():
            if s.isdigit():
                hold.loc[0,'Residue'] = s
            else:
                hold.loc[0,'Chain'] = s
                info = info.append(hold)
            
    info = info.set_index('Chain')
    info = info.sort('Residue',ascending=True)
    pivotlist = ''
    for indx, info in info.iterrows():
        pivotlist = pivotlist + ',' + info['Residue'] + indx
    ####ADD... turn all data in info dataframe into comma delimited <resnum><chain> pairs
    pivotlist = pivotlist[1:]
    return pivotlist

#Reads resfile and returns mutation position+chain and type
def resfile_stuff(pdb_filepath):
    resfile = read_mutations_resfile("/Users/jameslucas/Kortemme_Rotation/"+pdb_filepath)
    for i in resfile:
        position = i[0]
        chain = i[1]
        mut_to = i[3]
    return position, chain, mut_to
    
#Prints CMD input with PDBID, associated mutation, and pivot residues
def bash(json):
    for indx, chain_path in json.iterrows():
        pdb_file = chain_path['PDB File Path']
        chaintomove = chain_path['Chaintomove']
        
        #Removes PDB file from path, saves in variable data_dir
        pdb_file_parse = re.sub("/",' ', str(pdb_file))
        data, filenum, pdbname = pdb_file_parse.split()
        data_dir = data + "/" + filenum
        
        #Assigns function output to variables for bash input (pivot_residues, target, new_res)
        target, chain, new_res = resfile_stuff(data_dir)
        pivot_residues = neighbors_list(data_dir, pdb_file)
        
####All the variables and stuff for printing out the bash script
        print "~/rosetta_src_2016.08.58479_bundle/main/source/bin/rosetta_scripts.linuxgccrelease \
-s ~/%s ~/DDG_Test.xml -ignore_unrecognized_res -out:path:pdb ~/%s -nstruct 100 -parser:script_vars \
target=%s new_res=%s pivot_residues=%s" %(pdbname, pdbname, target, new_res, pivot_residues)
        
json = json_parser()
bash(json)

~/rosetta_src_2016.08.58479_bundle/main/source/bin/rosetta_scripts.linuxgccrelease -s ~/1TM1_EI.pdb ~/DDG_Test.xml -ignore_unrecognized_res -out:path:pdb ~/1TM1_EI.pdb -nstruct 100 -parser:script_vars target=61 new_res=A pivot_residues=155E,187E,189E,218E,220E,221E,59I,60I,61I,62I,63I
